In [48]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import h5py
from scipy.stats import entropy
from scipy.spatial.distance import pdist, squareform
import scipy.cluster.hierarchy as _sch
from activ import load_data
from activ.readfile import load_preprocessed, TrackTBIFile, encode, impute
import pandas as pd
from math import ceil
from matplotlib import cm
from sklearn import preprocessing
from collections import Counter
from activ.data_normalization import data_normalization

In [3]:
subdata = load_data()
subdata_oc = subdata.outcomes
subdata_bm = subdata.biomarkers
oc_features = subdata.outcome_features
bm_features = subdata.biomarker_features
sub_features = np.hstack((subdata.biomarker_features, subdata.outcome_features))
ids = subdata.patient_ids

In [4]:
fulldata = load_preprocessed()

In [5]:
gose12 = fulldata[np.logical_or(fulldata.GOSE_OverallScore6M == 1.0, fulldata.GOSE_OverallScore6M == 2.0)]

In [6]:
len(gose12)

29

In [7]:
gose12_ids = gose12.index

In [8]:
# one-hot encoding oc
gose12['GOSE_Reponder6M_Patient.alone'] = np.zeros(29)
gose12['GOSE_Reponder6M_Patient.plus.relative'] = [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]


/Users/ahyeon/anaconda3/envs/activ/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ahyeon/anaconda3/envs/activ/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# one-hot encoding bm

gose12['cause_Assault'] = [0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0]
gose12['cause_Fall'] = [1,1,1,1,0,1,1,0,1,1,0,1,0,1,1,1,1,1,1,0,1,1,1,1,0,0,0,1,0]
gose12['cause_MV.car.'] = [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1]
gose12['cause_MV.cyclist.ped.'] = [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]
gose12['cause_MV.motorcycle.'] = np.zeros(29)
gose12['cause_Other'] = np.zeros(29)
gose12['cause_Struck.By'] = np.zeros(29)
gose12['DispER_Discharge.Home'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
gose12['DispER_ICU.Admit'] = [0,1,0,1,1,0,1,0,1,1,1,1,0,1,0,0,1,1,0,1,1,1,0,1,0,0,0,0,0]
gose12['DispER_Operating.Room'] = [0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0]
gose12['DispER_Stepdown.Admit'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0]
gose12['DispER_Ward Admit'] = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0]
gose12['PatientTypeCoded_ED.Discharge'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
gose12['PatientTypeCoded_ICU.Admit'] = [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0]
gose12['PatientTypeCoded_Ward.Admit'] = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0]
gose12['PresArrivalMethod_Ambulance'] = [1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1]
gose12['PresArrivalMethod_Helicopter'] = [0,1,0,1,1,0,0,1,1,1,1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0]
gose12['PresArrivalMethod_Walk.in'] = np.zeros(29)
gose12['RACE_3CAT_Black'] = np.zeros(29)
gose12['RACE_3CAT_Other'] = np.zeros(29)
gose12['RACE_3CAT_White'] = np.ones(29)
gose12['injurytype_Closed'] = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1]
gose12['injurytype_Penetrating'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]

/Users/ahyeon/anaconda3/envs/activ/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ahyeon/anaconda3/envs/activ/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/ahyeon/anaconda3/envs/activ/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [10]:
keep_cols = np.asarray(list(set(sub_features).intersection(gose12.columns)))

In [19]:
247+29

276

In [12]:
gose12_oc_features = np.asarray(list(set(keep_cols).intersection(oc_features)))
gose12_bm_features = np.asarray(list(set(keep_cols).intersection(bm_features)))

In [13]:
drop_cols = list(set(gose12.columns).difference(keep_cols))

In [14]:
gose12.shape

(29, 943)

In [15]:
gose12 = gose12.drop(columns = drop_cols)

In [17]:
gose12_bm = gose12.drop(columns = gose12_oc_features)
gose12_oc = gose12.drop(columns = gose12_bm_features)

In [18]:
gose12_bm.shape, gose12_oc.shape

((29, 235), (29, 177))

In [20]:
pd.DataFrame(gose12_bm).to_csv("/Users/ahyeon/Desktop/activ_misc/gose12_bm.csv")
pd.DataFrame(gose12_oc).to_csv("/Users/ahyeon/Desktop/activ_misc/gose12_oc.csv")

In [69]:
subdata_bm = pd.DataFrame(subdata_bm, columns=bm_features)
subdata_oc = pd.DataFrame(subdata_oc, columns=oc_features)

In [70]:
for ii,f in enumerate(bm_features):
    if (1. in np.unique(subdata_bm[f])) and (2. in np.unique(subdata_bm[f])) and len(np.unique(subdata_bm[f]))==2:
        subdata_bm[f] = data_normalization(subdata_bm[f], 'positive')

In [73]:
for ii,f in enumerate(oc_features):
    if (1. in np.unique(subdata_oc[f])) and (2. in np.unique(subdata_oc[f])) and len(np.unique(subdata_oc[f]))==2:
        subdata_oc[f] = data_normalization(subdata_oc[f], 'positive')

In [ ]:
pd.DataFrame(gose12_bm).to_csv("/Users/ahyeon/Desktop/activ_misc/gose12_bm.csv")
pd.DataFrame(gose12_oc).to_csv("/Users/ahyeon/Desktop/activ_misc/gose12_oc.csv")

In [225]:
def encode(df, get_binvars=False):
    import pandas as pd
    from sklearn.preprocessing import LabelEncoder, LabelBinarizer
    cols = dict()
    enc = LabelEncoder()
    lb = LabelBinarizer()
    binvars = dict()
    for colname, coltype in df.dtypes.items():
        if coltype == object:
            newcol = enc.fit_transform(df[colname].astype(str))
            cols[colname] = newcol
            binvars[colname] = enc.classes_
        else:
            cols[colname] = df[colname]
    newdf = pd.DataFrame(cols, index=df.index)
    if get_binvars:
        return newdf, binvars
    else:
        return newdf

In [303]:
gose12_bm_redo = pd.read_csv('/Users/ahyeon/Desktop/activ_misc/gose12_bm_redo.csv', header=None)

In [309]:
gose12_bm_redo = gose12_bm_redo[1:]

In [310]:
redo_id = gose12_bm_redo.iloc[:,0]

In [312]:
age = gose12_bm_redo.iloc[:,1]

In [314]:
bm = gose12_bm_redo.drop(columns=0)

In [318]:
gose12_bm_redo = encode(bm)

In [319]:
gose12_bm_redo

,2,3,4,5,6,7,8,9,10,11,...,226,227,228,229,230,231,232,233,234,235
1,1,0,0,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,1,0
3,1,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
4,1,0,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,1,0
5,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
6,1,0,0,0,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,1,0
7,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
8,1,0,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,1,0
9,1,0,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,1,0
10,0,0,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,1,0


In [320]:
df = gose12_bm_redo.insert(0,1,age)

In [321]:
gose12_bm_redo.shape

(29, 235)

In [322]:
gose12_oc_redo = pd.read_csv('/Users/ahyeon/Desktop/activ_misc/gose12_oc_redo.csv', header=None)

In [323]:
gose12_oc_redo.shape

(30, 178)

In [324]:
gose12_oc_redo = gose12_oc_redo[1:].drop(columns=0)

In [325]:
gose12_bm_redo.shape, gose12_oc_redo.shape

((29, 235), (29, 177))

In [330]:
final_gose12_bm = np.vstack((subdata_bm, gose12_bm_redo))
final_gose12_oc = np.vstack((subdata_oc, gose12_oc_redo))
final_ids = np.hstack((ids, redo_id))

In [353]:
final_gose12_bm = np.asarray(final_gose12_bm, dtype=float)
final_gose12_oc = np.asarray(final_gose12_oc, dtype=float)

In [354]:
final_gose12_bm.shape, final_gose12_oc.shape

((276, 235), (276, 177))

In [358]:
from sklearn.utils import shuffle
bm_shuffle, oc_shuffle, id_shuffle = shuffle(final_gose12_bm,final_gose12_oc,final_ids)

In [359]:
final_gose12_bm

array([[24.,  1.,  0., ...,  0.,  0.,  0.],
       [45.,  1.,  0., ...,  0.,  0.,  0.],
       [28.,  1.,  0., ...,  0.,  0.,  0.],
       ...,
       [51.,  1.,  0., ...,  0.,  1.,  0.],
       [41.,  1.,  0., ...,  0.,  1.,  0.],
       [46.,  0.,  0., ...,  0.,  1.,  0.]])

In [360]:
fname_shuffled = '/Users/ahyeon/Desktop/activ_misc/gose12_shuffled_final.h5'
TrackTBIFile.write(fname_shuffled, bm_shuffle, oc_shuffle, biomarker_features = bm_features, outcome_features=oc_features, patient_ids=id_shuffle)